In [1]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.3 MB/s eta 0:00:00


In [2]:
import os
from groq import Groq
from google.colab import userdata
client = Groq(api_key=userdata.get('groq_lama'),)

In [3]:
!pip install pandas

In [4]:
import pandas as pd
def read_prompts(file):
    try:
        df = pd.read_csv(file, sep='\t', encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file, sep='\t', encoding='ISO-8859-1')
    prompts = []
    for index, row in df.iterrows():
        prompt = f"""
        You are given a multiple-choice question (MCQ) from a Cyber Threat Intelligence (CTI) knowledge benchmark dataset. /
        Your task is to choose the best option among the four provided. /
        Return your answer as a single uppercase letter: A, B, C, or D without any explication or additional text or introduction. /
        **Question:** {row['Question']} /
        **Options:** A) {row['Option A']} B) {row['Option B']} C) {row['Option C']} D) {row['Option D']} /
        **Important:** Your answer should contain only the single letter corresponding to the best option, with no additional text or introduction. /
        """
        prompts.append(prompt)
    return df, prompts

In [5]:
import time
import requests
from http.client import RemoteDisconnected
from google.api_core.exceptions import TooManyRequests, InternalServerError
import pandas as pd
import os
import re

def execute_prompts(df, prompts):
    results = []
    correct_answers = df['GT'].tolist()
    total_correct = 0
    max_retries = 2500  # Maximum number of retries for connection errors

    for i, prompt in enumerate(prompts):
        retries = 0
        while retries < max_retries:
            try:
                completion = client.chat.completions.create (
                       model="mixtral-8x7b-32768",
                       messages=[
                           {"role": "user", "content": prompt},
                        ],
                       temperature= 0.9,
                       max_tokens = 1,
                       top_p=1,
                       stream=True,
                       stop=None,
                )
                full_response = ""
                for chunk in completion:
                    # Ensure the chunk has the expected attributes
                    if hasattr(chunk, 'choices') and chunk.choices:
                        if hasattr(chunk.choices[0], 'delta') and hasattr(chunk.choices[0].delta, 'content'):
                            content = chunk.choices[0].delta.content
                            if content is not None:
                                full_response += str(content)
                                print(content)  # Print each chunk of response

                if full_response.strip() == "":
                    print(f"Response was empty. Retrying prompt {i+1}...")
                    time.sleep(5)  # Wait before retrying
                    retries += 1
                    continue  # Retry if the response is empty

                result = full_response.strip()
                filtered_result = [r for r in result if re.match(r'^[ABCD]$', r)]
                for j in range(len(correct_answers)):
                  if j < len(filtered_result):
                    results.append(filtered_result[j])
                    print(filtered_result[j])
                    if filtered_result[j] == correct_answers[i]:
                      total_correct += 1  # Append the filtered letter if it exists
                  else:
                    results.append("X")

                # Add a delay between requests to avoid hitting the rate limit
                time.sleep(2)  # Adjust the sleep duration as needed
                break  # Exit the retry loop if successful
            except TooManyRequests:
                print(f"Too many requests. Retrying prompt {i+1} after a delay...")
                time.sleep(60)  # Wait for 60 seconds before retrying
            except (ConnectionError, RemoteDisconnected) as e:
                print(f"Connection error: {e}. Retrying prompt {i+1}...")
                retries += 1
                time.sleep(2)  # Adjust the sleep duration as needed
            except ValueError as e:
                print(f"ValueError: {e}. Skipping prompt number {i+1}.")
                results.append("ERROR")
                break  # Skip this prompt
            except InternalServerError as e:
                print(f"InternalServerError: {e}. Skipping prompt number {i+1}.")
                results.append("ERROR")
                break  # Skip this prompt

        if retries == max_retries:
            print(f"Failed to process prompt {i+1} after {max_retries} retries.")
            results.append("ERROR")

    df['Mistral 8x7b'] = results

    # Calculate the total correct answers
    accuracy = total_correct / len(correct_answers)
    output_file = "/content/Mistral_mcq.tsv"

    if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
        try:
            existing_df = pd.read_csv(output_file, sep='\t', encoding='utf-8')
        except UnicodeDecodeError:
            existing_df = pd.read_csv(output_file, sep='\t', encoding='latin1')
        combined_df = pd.concat([existing_df, df], ignore_index=True)
    else:
        combined_df = df


    # Save the updated file with Gemini's responses
    combined_df.to_csv(output_file, sep='\t', index=False)

    # Print the total correct answers
    print(f'Total correct responses by Mistral: {total_correct}/{len(correct_answers)}')
    # Print the final accuracy
    print(f'Final accuracy: {accuracy:.2%}')



In [7]:
df, prompts = read_prompts("/content/mydataset_mcq.tsv")
execute_prompts(df, prompts)


A
A

The

D
D

The
 primary

B
)
B

D
D

C
C

B
)
B

D
D

The
 correct

B
)
B

B
B

B
B

A
A

B
)
B

B
B

The

D
)
D

B
)
B

D
D

B
B

B
)
B

D
D

B
)
B

D
)
D

D
)
D

C
)
C

A
A

The
 correct

B


B

C
)
C

The
 correct

D
D

B
)
B

D
)
D

D
D

B
)
B

A
 
A

B


B

B
)
B

B


B

A
A

The

A
A

C
)
C

A
A

C
C

D
)
D

The

D
)
D

B
B

B


B

D
D

A
)
A

C
C

A
A

A
A

A


A

A
A

B
)
B

B


B

D
)
D

A
A

B
)
B

A
A

C


C

A
)
A

The

A
 
A

B
)
B

A
A

The
 best

A
A

C
C

A
A

The
 correct

A
A

D


D

A
A

D
D

A


A

A
A

A
A

A
A

A
A

A
A

B
B

B


B

A
A

B


B

A
A

B
B

A
A

A
A

B
B

C
C

D
D

B
B

A
 
A

D


D

A
 
A

A
)
A

C
)
C

A
A

C
)
C

D
)
D

B


B

C


C

The
 correct

B
B

D
D

C
)
C

A


A

A
A

C


C

D
)
D

D
D

The
 technique

D
)
D

C
C

C
)
C

D
)
D

A
A

A
A

B


B

B
)
B

A
A

B
)
B

A
A

C
C

A
A

B
B

D
)
D

The
 correct

A
A

A
A

A
A

D
)
D

A
,
A

D
D

A
A

C
C

The

C
)
C

A
A

B
)
B

D
D

B
B

The
 primary

A
A

B
B

A
)
A

B
B

The
 advers

A
)
A


ValueError: Length of values (2250000) does not match length of index (1500)

In [6]:
import pandas as pd
import time
from http.client import RemoteDisconnected
from google.api_core.exceptions import TooManyRequests, InternalServerError
import os

def read_prompts_rcm(file):
    try:
        df = pd.read_csv(file, sep='\t', encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file, sep='\t', encoding='ISO-8859-1')

    prompts = []
    for index, row in df.iterrows():
        prompt = f"""
        You are a cybersecurity expert specializing in cyber threat intelligence. \
        Analyze the following CVE description and map it to the appropriate CWE. \
        your response should contains only the CWE ID. \
        CVEDescription: {row['Description']}
        """
        prompts.append(prompt)
    return df, prompts

def execute_prompts_rcm(df, prompts):
    results = []
    correct_answers = df['GT'].tolist()
    total_correct = 0
    max_retries = 2500  # Maximum number of retries for connection errors

    for i, prompt in enumerate(prompts):
        success = False
        retries = 0
        while not success and retries < max_retries:
            try:
                completion = client.chat.completions.create(
                    model="mixtral-8x7b-32768",
                    messages=[
                        {"role": "user", "content": prompt},
                    ],
                    temperature=0.9,
                    max_tokens=5,
                    top_p=1,
                    stream=True,
                    stop=None,
                )

                full_response = ""
                for chunk in completion:
                    if hasattr(chunk, 'choices') and chunk.choices:
                        if hasattr(chunk.choices[0], 'delta') and hasattr(chunk.choices[0].delta, 'content'):
                            content = chunk.choices[0].delta.content
                            if content is not None:
                                full_response += str(content)
                                print(content)  # Print each chunk of response

                if full_response == "":
                    print(f"No content in response. Retrying prompt {i+1}...")
                    retries += 1
                    time.sleep(2)  # Short sleep before retry
                    continue  # Retry the prompt

                result = full_response.strip()  # Convert to string and strip spaces

                # Extract the last line starting with "CWE-"
                last_line = ""
                for line in reversed(result.split("\n")):
                    if line.strip().startswith("CWE-"):
                        last_line = line.strip()
                        break

                if last_line == "":
                    print(f"No CWE found in response. Retrying prompt {i+1}...")
                    retries += 1
                    time.sleep(2)  # Short sleep before retry
                    continue  # Retry the prompt

                results.append(result)
                print(results)  # Print accumulated results so far

                if last_line.upper() == correct_answers[i].strip().upper():
                    total_correct += 1

                success = True  # Exit the retry loop if successful
                time.sleep(2)  # Delay between requests to avoid hitting the rate limit
            except TooManyRequests:
                print(f"Too many requests. Retrying prompt {i+1} after a delay...")
                time.sleep(60)  # Wait for 60 seconds before retrying
            except (ConnectionError, RemoteDisconnected) as e:
                print(f"Connection error: {e}. Retrying prompt {i+1}...")
                retries += 1
                time.sleep(2)  # Adjust the sleep duration as needed
            except ValueError as e:
                print(f"ValueError: {e}. Skipping prompt number {i+1}.")
                results.append("ERROR")
                break  # Skip this prompt
            except InternalServerError as e:
                print(f"InternalServerError: {e}. Skipping prompt number {i+1}.")
                results.append("ERROR")
                break  # Skip this prompt

        if not success:
            print(f"Failed to process prompt {i+1} after {max_retries} retries.")
            results.append("ERROR")

    df['Mistral Response'] = results

    # Calculate the total correct answers
    accuracy = total_correct / len(correct_answers)
    output_file = "/content/output_file_rcm.tsv"

    if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
        existing_df = pd.read_csv(output_file, sep='\t', encoding='utf-8')
        combined_df = pd.concat([existing_df, df], ignore_index=True)
    else:
        combined_df = df

    # Save the updated file with Gemini's responses
    combined_df.to_csv(output_file, sep='\t', index=False)

    # Print the total correct answers
    print(f'Total correct responses by Mistral: {total_correct}/{len(correct_answers)}')
    # Print the final accuracy
    print(f'Final accuracy: {accuracy:.2%}')

# Supposons que vous ayez un DataFrame 'df' et une liste de prompts 'prompts'
df, prompts = read_prompts_rcm("/content/mydataset_rcm.tsv")
execute_prompts_rcm(df, prompts)



C
WE
-
3
1
2
['CWE-312']

C
WE
-
3
8
4
['CWE-312', 'CWE-384']

C
WE
-
2
0
9
['CWE-312', 'CWE-384', 'CWE-209']

C
WE
-
2
0
0
['CWE-312', 'CWE-384', 'CWE-209', 'CWE-200']

C
WE
-
2
6
['CWE-312', 'CWE-384', 'CWE-209', 'CWE-200', 'CWE-26']

C
WE
-
2
0
['CWE-312', 'CWE-384', 'CWE-209', 'CWE-200', 'CWE-26', 'CWE-20']

C
WE
-
2
0
['CWE-312', 'CWE-384', 'CWE-209', 'CWE-200', 'CWE-26', 'CWE-20', 'CWE-20']

C
WE
-
2
5
['CWE-312', 'CWE-384', 'CWE-209', 'CWE-200', 'CWE-26', 'CWE-20', 'CWE-20', 'CWE-25']

C
WE
-
1
4
:
['CWE-312', 'CWE-384', 'CWE-209', 'CWE-200', 'CWE-26', 'CWE-20', 'CWE-20', 'CWE-25', 'CWE-14:']

C
WE
-
2
0
['CWE-312', 'CWE-384', 'CWE-209', 'CWE-200', 'CWE-26', 'CWE-20', 'CWE-20', 'CWE-25', 'CWE-14:', 'CWE-20']

C
WE
-
2
0
:
['CWE-312', 'CWE-384', 'CWE-209', 'CWE-200', 'CWE-26', 'CWE-20', 'CWE-20', 'CWE-25', 'CWE-14:', 'CWE-20', 'CWE-20:']

C
WE
-
2
2
['CWE-312', 'CWE-384', 'CWE-209', 'CWE-200', 'CWE-26', 'CWE-20', 'CWE-20', 'CWE-25', 'CWE-14:', 'CWE-20', 'CWE-20:', 'CWE-22']

C
W

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01j44ntbf7fhe9t5126gh0xrvg` on : Limit 500000, Used 499970, Requested 765. Please try again in 2m7.0076s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}